In [1]:
import dask
from dask.diagnostics import ProgressBar
#from dask.distributed import Client
from clisops.core.subset import create_weight_masks
import geopandas as gpd
from pathlib import Path
from siphon.catalog import TDSCatalog
import xarray as xr
import xclim as xc
import xscen as xs
dask.config.set(num_workers=8)
ProgressBar().register()

2023-05-01 15:57:52,419 - pint.util - WARNING - Redefining 'year' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,421 - pint.util - WARNING - Redefining 'yr' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,424 - pint.util - WARNING - Redefining 'C' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,428 - pint.util - WARNING - Redefining 'd' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,431 - pint.util - WARNING - Redefining 'h' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,434 - pint.util - WARNING - Redefining 'degrees_north' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,438 - pint.util - WARNING - Redefining 'degrees_east' (<class 'pint.delegates.txt_defparser.plain.UnitDefinition'>)
2023-05-01 15:57:52,441 - pint.util - WARNING - Redefining '[speed]' (<class 'pint.delegates.txt_d

In [2]:
# Create Catalog
cat6 = TDSCatalog('https://pavics.ouranos.ca/testthredds/catalog/testdatasets/CanDCS-U6/catalog.xml')
cat5 = TDSCatalog('https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/catalog/datasets/simulations/bias_adjusted/cmip5/pcic/bccaqv2/catalog.xml')

In [3]:
hausfat = [
    "BCC-CSM2-MR", "FGOALS-g3", "CMCC-ESM2", "CNRM-ESM2-1", "ACCESS-CM2", "ACCESS-ESM1-5",
    "MPI-ESM1-2-HR", "INM-CM5-0", "MIROC6", "MPI-ESM1-2-LR", "MRI-ESM2-0", "NorESM2-LM", "KACE-1-0-G", "GFDL-ESM4"
]
sgmembs = [
    'CanESM2', 'CMCC-CMS', 'ACCESS1-3', 'BNU-ESM', 'INM-CM4',
    'IPSL-CM5A-LR', 'IPSL-CM5B-LR', 'HadGEM2-CC', 'MPI-ESM-LR', 'NorESM1-M', 'GFDL-ESM2M'
]
pcic12 = [
    ("ACCESS1-0" , "r1i1p1"),
    ("CCSM4" , "r2i1p1"),
    ("CNRM-CM5" , "r1i1p1"),
    ("CSIRO-Mk3-6-0" , "r1i1p1"),
    ("CanESM2" , "r1i1p1"),
    ("GFDL-ESM2G" , "r1i1p1"),
    ("HadGEM2-CC" , "r1i1p1"),
    ("HadGEM2-ES" , "r1i1p1"),
    ("MIROC5" , "r3i1p1"),
    ("MPI-ESM-LR" , "r3i1p1"),
    ("MRI-CGCM3" , "r1i1p1"),
    ("inmcm4" , "r1i1p1")
]


probreg = dict(
    name="prob-reg",
    method="bbox",
    bbox=dict(
        lat_bnds=[45, 47],
        lon_bnds=[-75, -70]
    ),
)
QC = dict(
    name="QC-rdrs",
    method="bbox",
    bbox=dict(
        lon_bnds=[-82.7, -55],
        lat_bnds=[41, 63]
    )
)

okreg = dict(
    name="ok-reg",
    method="bbox",
    bbox=dict(
        lat_bnds=[48, 50],
        lon_bnds=[-75, -70]
    ),
)
root = Path('/exec/pbourg/ESPO-G/indsanal/')
#regions = gpd.read_file('/exec/pbourg/PC/20230226/regions_simplified_admin.geojson')
#estrie = regions.iloc[[29]]

In [28]:
template = 'CanDCS-U6_CMIP6_ScenarioMIP_{model}_{model}_ssp245_r1i1p1f1_CAN'

for name, dsi in cat6.datasets.items():
    parts = name.split('_')
    if not ('ssp245' in parts[3] and parts[2] in hausfat):
        continue
        
    dsid = template.format(model=parts[2])   
    print('Processing', dsid)
    outfile = Path(f'/exec/pbourg/ESPO-G/indsanal/day_{dsid}_QC.zarr')
    if not outfile.is_dir():
        print('Daily')
        dsi = xr.open_dataset(
            dsi.access_urls['OPENDAP'],
            chunks={'lat': 50, 'lon': 50},
        )
        ds = xs.extract.clisops_subset(dsi, region=QC)
        ds = xr.concat(
            [ds.sel(time=slice('1991', '2020')), ds.sel(time=slice('2071', '2100'))],
            'time'
        )
        #ds.cf.chunk({'X': -1, 'Y': -1})
        ds.to_zarr(outfile)

    outfile2 = outfile.parent / outfile.name.replace('day_', 'mon_')
    if outfile2.is_dir():
        print('Done')
        continue
    ds = xr.open_zarr(outfile)
    out = xr.concat(
        [
            xc.atmos.daily_freezethaw_cycles(ds=ds.sel(time=sel), freq='MS')
            for sel in [slice('1991', '2020'), slice('2071', '2100')]
        ],
        'time'
    ).to_dataset().chunk({'time': -1})
    print('Monthly')
    out.to_zarr(outfile2)


Processing CanDCS-U6_CMIP6_ScenarioMIP_NorESM2-LM_NorESM2-LM_ssp245_r1i1p1f1_CAN
Monthly
[########################################] | 100% Completed | 536.56 s
Processing CanDCS-U6_CMIP6_ScenarioMIP_MRI-ESM2-0_MRI-ESM2-0_ssp245_r1i1p1f1_CAN
Daily
[########################################] | 100% Completed | 23m 59s
Monthly
[########################################] | 100% Completed | 573.67 s
Processing CanDCS-U6_CMIP6_ScenarioMIP_MPI-ESM1-2-LR_MPI-ESM1-2-LR_ssp245_r1i1p1f1_CAN
Daily
[########################################] | 100% Completed | 23m 45s
Monthly
[########################################] | 100% Completed | 531.89 s
Processing CanDCS-U6_CMIP6_ScenarioMIP_MPI-ESM1-2-HR_MPI-ESM1-2-HR_ssp245_r1i1p1f1_CAN
Daily
[########################################] | 100% Completed | 23m 37s
Monthly
[########################################] | 100% Completed | 558.48 s
Processing CanDCS-U6_CMIP6_ScenarioMIP_MIROC6_MIROC6_ssp245_r1i1p1f1_CAN
Daily
[########################################

In [23]:
dsi.sel(lat=slice(41, 63)).tasmin.chunks[1]

(50, 50, 50, 50, 50, 14)

In [5]:
template = 'CanDCS-U5_CMIP5_ScenarioMIP_{model}_{model}_rcp45_r1i1p1f1_CAN'
for name, dsi in cat5.datasets.items():
    parts = name.split('_')
    if not ('rcp45' in parts[2]): # and (parts[1], parts[3]) in pcic12):
        print("not", name)
        continue
        
    dsid = template.format(model=parts[1])
    print('Processing', dsid)
    outfile = Path(f'/exec/pbourg/ESPO-G/indsanal/day_{dsid}_prob-reg.zarr')
    if not outfile.is_dir():
        print('Daily')
        ds = xr.open_dataset(
            dsi.access_urls['OPENDAP'],
            chunks={'lat': 64, 'lon': 64},
        )
        ds = xs.extract.clisops_subset(ds, region=probreg)
        ds.cf.chunk({'X': -1, 'Y': -1}).to_zarr(outfile)

    continue
    outfile2 = outfile.parent / outfile.name.replace('day_', 'yr_')
    if outfile2.is_dir():
        print('Done')
        continue
    ds = xr.open_zarr(outfile)
    out = xr.merge(
        [
            xc.atmos.daily_freezethaw_cycles(ds=ds, freq='MS'),
            xc.atmos.daily_temperature_range(ds=ds, freq='MS'),
            create_weight_masks(ds, estrie).squeeze('geom', drop=True).rename('mask')
        ]
    ).chunk({'time': -1})
    print('Monthly')
    out.to_zarr(outfile2)

not day_bcc-csm1-1_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_bcc-csm1-1_bcc-csm1-1_rcp45_r1i1p1f1_CAN
Daily


/home/pbourg/Projets/xscen/xscen/extract.py:73: FutureWarning: clisops_subset is deprecated and will not be available in future versions. Use xscen.spatial.subset instead.


[########################################] | 100% Completed | 85.53 s
not day_bcc-csm1-1_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
not day_bcc-csm1-1-m_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_bcc-csm1-1-m_bcc-csm1-1-m_rcp45_r1i1p1f1_CAN
Daily


/home/pbourg/Projets/xscen/xscen/extract.py:73: FutureWarning: clisops_subset is deprecated and will not be available in future versions. Use xscen.spatial.subset instead.


[########################################] | 100% Completed | 11m 10s
not day_bcc-csm1-1-m_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
not day_NorESM1-M_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_NorESM1-M_NorESM1-M_rcp45_r1i1p1f1_CAN
not day_NorESM1-M_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
not day_NorESM1-ME_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_NorESM1-ME_NorESM1-ME_rcp45_r1i1p1f1_CAN
Daily


...

[########################################] | 100% Completed | 85.99 s
not day_CNRM-CM5_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
not day_CESM1-CAM5_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_CESM1-CAM5_CESM1-CAM5_rcp45_r1i1p1f1_CAN
Daily


/home/pbourg/Projets/xscen/xscen/extract.py:73: FutureWarning: clisops_subset is deprecated and will not be available in future versions. Use xscen.spatial.subset instead.


[########################################] | 100% Completed | 10m 17s
not day_CESM1-CAM5_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
not day_CCSM4_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_CCSM4_CCSM4_rcp45_r1i1p1f1_CAN
Daily


/home/pbourg/Projets/xscen/xscen/extract.py:73: FutureWarning: clisops_subset is deprecated and will not be available in future versions. Use xscen.spatial.subset instead.


[########################################] | 100% Completed | 589.55 s
not day_CCSM4_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
not day_BNU-ESM_historical+rcp85_r1i1p1_19500101-21001231_bccaqv2.ncml
Processing CanDCS-U5_CMIP5_ScenarioMIP_BNU-ESM_BNU-ESM_rcp45_r1i1p1f1_CAN
not day_BNU-ESM_historical+rcp26_r1i1p1_19500101-21001231_bccaqv2.ncml
